In [1]:
# Basis-Imports (oben)
import sys

# import gc
# import time
# import asyncio
import numpy as np

# Abschnitts-spezifisch (in der Zelle, wo es gebraucht wird)
# from numba import njit
# import pymc as pm

C:\Users\sebastian.bichler\OneDrive - IU International University of Applied Sciences\ProgProjects\IU_Python\IU_2026_EF_LE_BER_VC_Python\.venv\Lib\site-packages\arviz\__init__.py:39: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(


## 1. Memory Management: Der "Hausmeister" und das "Reinigungsteam"

Python nutzt ein Hybrid-System. Das Reference Counting (RC) löscht sofort, der Cyclic GC kümmert sich um die "toten Inseln".

In [ ]:
# import sys
# import gc


class Elephant:
    pass  # leere Klasse (Platzhalter)


e1 = Elephant()
print(f"Initialer Counter: {sys.getrefcount(e1) - 1}")

e2 = e1
print(f"Nach Zuweisung e2: {sys.getrefcount(e1) - 1}")

del e2
print(f"Nach del e2: {sys.getrefcount(e1) - 1}")

**Aufgabe:** Erstellen Sie zwei Instanzen einer Klasse Node, die sich gegenseitig referenzieren. Beweisen Sie mit gc.collect(), dass sie erst durch den zyklischen GC gelöscht werden.

**Hintergrund**: In C# würde der GC das Objekt irgendwann abholen. In Python bleibt es ohne den zyklischen GC ewig im Speicher, da der Reference Count bei 1 (gegenseitige Referenz) stehen bleibt.

## 2. GIL & Multi-Core: Das Nadelöhr

In C# nutzen wir alle Kerne. In Python verhindert das Global Interpreter Lock, dass zwei Threads gleichzeitig Python-Bytecode ausführen.

In [ ]:
# import time
# from threading import Thread
# from multiprocessing import Process


def heavy_task():
    # Eine CPU-intensive Berechnung
    return sum(i * i for i in range(10**7))


# Messen Sie die Zeit für 2x heavy_task() sequenziell vs. 2 Threads vs. 2 Prozesse.
# Spoileralarm: Threads bringen hier keinen Speedup!

# Messen Sie die Zeit für 2x cpu_work() sequenziell vs. 2 Threads vs. 2 Prozesse.
# Spoileralarm: Threads bringen hier keinen Speedup!

**Hintergrund**: In C# würde der GC das Objekt irgendwann abholen. In Python bleibt es ohne den zyklischen GC ewig im Speicher, da der Reference Count bei 1 (gegenseitige Referenz) stehen bleibt.

**Aufgabe:** Erstellen Sie zwei Instanzen einer Klasse Node, die sich gegenseitig referenzieren. Beweisen Sie mit gc.collect(), dass sie erst durch den zyklischen GC gelöscht werden.

## 3. Asyncio: Der flinke Kellner

Warum 10.000 Tasks besser sind als 10.000 Threads.

**Aufgabe**: Simuliere das gleichzeitige Abrufen von 50 Sensordaten-Paketen (I/O-bound), ohne 50 Sekunden warten zu müssen.

## 4. NumPy: Vektorisierung & SIMD

Warum for-Schleifen in Python "böse" sind (wenn es um Mathe geht).

In [ ]:
# import numpy as np

data = np.random.rand(10**7)

# Vergleichen Sie:
# 1. [x**2 for x in data] (Listen-Abstraktion)
# 2. np.square(data) (Vektorisierung)

**Hintergrund**: NumPy nutzt intern hochoptimierte C-Schleifen und SIMD-Register der CPU.

**Aufgabe**: Berechne das Quadrat von 1 Million Werten und vergleiche die Geschwindigkeit einer Python-Schleife mit einer NumPy-Operation.

## 5. Metaprogrammierung: Code, der Code schreibt

Wir nutzen Decorators, um unsere API-Aufrufe im Projekt zu schützen.

🧪 Experiment: Der @log_access Decorator

In [ ]:
def log_access(func):
    def wrapper(*args, **kwargs):
        print(f"Audit-Log: Zugriff auf {func.__name__}")
        return func(*args, **kwargs)

    return wrapper


@log_access
def get_elephant_secret():
    return "Elefanten vergessen nie."


print(get_elephant_secret())

## 6. Numba JIT: Python bei Lichtgeschwindigkeit

Wenn NumPy nicht reicht, kompilieren wir Python-Code zur Laufzeit in Maschinencode.

**Beispiel**: Die Monte-Carlo-Pi-Simulation zur Berechnung von Pi: π ≈ 4⋅Punkte im Quadrat / Punkte im Kreis

In [ ]:
from numba import njit
import random


@njit
def fast_pi(n):
    acc = 0
    for i in range(n):
        if random.random() ** 2 + random.random() ** 2 < 1.0:
            acc += 1
    return 4.0 * acc / n

**Aufgabe**: Messen Sie die Ausführungszeit von fast_pi(10**7) im Vergleich zu einer reinen Python-Implementierung der Monte-Carlo-Methode zur Berechnung von Pi.

## 7. Probabilistische Programmierung (PyMC)

Unsicherheit ist eine Information, kein Fehler. Wenden Sie die Bayesianische Schätzung an.

In [ ]:
import pymc as pm

with pm.Model() as model:
    # ir vermuten das Gewicht um 5000kg (Prior)
    weight = pm.Normal("weight", mu=5000, sigma=500)
    # ... Inferenz-Code hier ...

**Aufgabe**: Fügen Sie Beobachtungsdaten hinzu (z.B. gemessene Gewichte) und führen Sie eine Inferenz durch, um die Posterior-Verteilung des Gewichts zu erhalten.

## 8. Structural Design Patterns: Adapter

Dank Duck Typing sind Patterns in Python oft nur eine einfache Klasse ohne Interfaces.

🧪 Experiment: Der GPS-Adapter

In [ ]:
# --- DAS BESTEHENDE SYSTEM ---


class LegacyGPSSensor:
    """Der Standard-Sensor im Projekt."""

    def get_coordinates(self):
        return {"lat": 51.31, "lon": 12.37}


# --- DIE NEUE, INKOMPATIBLE KLASSE ---


class NewHighTechSensor:
    """Ein neuer Sensor eines Drittanbieters mit anderer API."""

    def fetch_raw_data(self):
        # Liefert Daten als String statt als Dictionary
        return "51.339, 12.373"

**Aufgabe**: Unser System erwartet GPS-Daten als Dictionary über die Methode get_coordinates(). Ein neuer Sensor-Typ liefert die Daten jedoch als Roh-String über eine Methode namens fetch_raw_data(). Baue einen Adapter, um den neuen Sensor ohne Änderung am restlichen Code zu integrieren.